# 走行

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson NANOを認識")
    I2C_BUSNUM = 1
    MODE = 0

In [ ]:
!echo "jetson" | sudo -S nvpmodel -m $MODE

In [ ]:
!echo "jetson" | sudo -S nvpmodel -q

In [ ]:
!echo "jetson" | sudo -S jetson_clocks

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
INITIAL_VALUE=300
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

STEERING_CH = 0
THROTTLE_CH = 1
direction = 0

pwm_front = 0
pwm_back = 0

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    pwm_stop = json_str["pwm_speed"]["stop"]
    pwm_front = json_str["pwm_speed"]["front"]
    pwm_back = json_str["pwm_speed"]["back"]
    pwm_left = json_str["pwm_steering"]["left"]
    pwm_center = json_str["pwm_steering"]["center"]
    pwm_right = json_str["pwm_steering"]["right"]

    
if pwm_front >= pwm_back:
    direction = 0
else:
    direction = 1
    
PCA9685.set_channel_value(STEERING_CH, pwm_center)
PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=30)

In [ ]:
import torch
import threading
from utils import preprocess
import subprocess
import cv2
import time
from torch2trt import TRTModule
import subprocess
import datetime

IMG_WIDTH=224

model_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_time_widget = ipywidgets.Label(description='作成日時：')
load_button = ipywidgets.Button(description='Load')
run_button = ipywidgets.Button(description='Run')
stop_button = ipywidgets.Button(description='Stop')
pwm_left_widget = ipywidgets.IntText(value=pwm_left,description='PWM 左')
pwm_center_widget = ipywidgets.IntText(value=pwm_center,description='PWM 中央')
pwm_right_widget = ipywidgets.IntText(value=pwm_right,description='PWM 右')
check_left_button = ipywidgets.Button(description='チェック左')
check_center_button = ipywidgets.Button(description='チェック中央')
check_right_button = ipywidgets.Button(description='チェック右')
pwm_stop_widget = ipywidgets.IntText(value=pwm_stop,description='PWM 停止')
name_widget = ipywidgets.Text(description='映像の保存先')
record_box = ipywidgets.Checkbox(False, description='録画')
speed_slider = ipywidgets.IntSlider(description='speed', min=0, max=100, step=1, value=0, orientation='horizontal')

record = False
running = False

def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

def handle(x):
    global pwm_right,pwm_left,STEERING_CH,PCA9685
    x = map_rc(x, 224, 0, pwm_right, pwm_left)
    PCA9685.set_channel_value(STEERING_CH, x)
    
def live():
    global running,camera,IMG_WIDTH,record,model_trt,preprocess,pwm_stop,save_dir,num,cv2,count,speed_slider
    
    count = 1
    num = 1
    
    while running:
        image = camera.read()
        if record == True:
            remarked_img = image.copy()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        y = float(output[1])
        speed = float(output[3])
        x = int(IMG_WIDTH * (x / 2.0 + 0.5))
        y = int(IMG_WIDTH * (y / 2.0 + 0.5))
        speed = int(IMG_WIDTH * (speed / 2.0 + 0.5))
        handle(x)
        
        if direction == 0:
            pwm_speed = pwm_stop + speed_slider.value
        else:
            pwm_speed = pwm_stop - speed_slider.value
        PCA9685.set_channel_value(THROTTLE_CH, pwm_speed)
        
        #if speed > 112:
        #    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop+55)
        #elif speed <= 112:
        #    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop+55)

        if record == True:
            if num % 2 == 0:
                name = "0_0_{:0=5}.jpg".format(num)
                image_path = os.path.join(save_dir, name)
                cv2.imwrite(image_path, remarked_img)
            num += 1
        count += 1
        
        #write_log("Count:" + str(count))
        
def run(change):
    global running,execute_thread,name_widget,save_dir,start_time
    
    if running == False:
        if record == True:
            if name_widget.value != "":
                save_dir = "run/" + name_widget.value + "/xy/" 
                if not os.path.exists(save_dir):
                    subprocess.call(['mkdir', '-p', save_dir])
                write_log(save_dir + "にデータを保存します。")            

                running = True
                execute_thread = threading.Thread(target=live)
                execute_thread.start()
                start_time = time.time()
                write_log("AIが起動しました。")
                if direction == 0:
                    pwm_speed = pwm_stop + speed_slider.value
                else:
                    pwm_speed = pwm_stop - speed_slider.value
                write_log("Speed:" + str(pwm_speed) + "で走行開始します。")
            else:
                write_log("映像の保存先を入力してください。")
        else:
            running = True
            execute_thread = threading.Thread(target=live)
            execute_thread.start()
            start_time = time.time()
            write_log("AIが起動しました。") 
            if direction == 0:
                pwm_speed = pwm_stop + speed_slider.value
            else:
                pwm_speed = pwm_stop - speed_slider.value
            write_log("Speed(PWM):" + str(pwm_speed) + "で走行開始します。")
        
    
def stop(change):
    global running,execute_thread,end_time,start_time,count,pwm_stop
    if running == True:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        try:
            end_time = time.time() - start_time
            fps = count/int(end_time)
            process_time = int((end_time/count)*1000)
        except:
            fps = -1
            process_time = -1
        
        write_log("AIを停止しました。")
        write_log("処理結果:FPS: " + str(round(fps,2)) + ",処理回数: " + str(count) + ",　処理時間(1回平均値): " + str(process_time) + " ms")
        running = False
        execute_thread.join()
    else:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        write_log("現在AIは動いていません。")

def load_model(change):
    global model_trt,model_widget
    try:
        write_log(model_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_trt = TRTModule()
        model_trt.load_state_dict(torch.load(model_widget.value))
        write_log(model_widget.value + "の読込に成功しました。")
    except:
        write_log("[Error]" + model_widget.value + "の読込に失敗しました。")
        
def model_list(change):
    global model_widget
    try:
        files = glob.glob('./model_trt/*.pth', recursive=True)
        model_widget.options = files
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        model_time_widget.value = f'作成日時：{s}'
    except:
        model_widget.options = []
model_list("list")

def change_file(change):
    file = model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_time_widget.value = f'作成日時：{s}'
model_widget.observe(change_file, names='value')

def on_video(change):
    global record
    record^=True

def check_left(change):
    global pwm_left,STEERING_CH
    PCA9685.set_channel_value(STEERING_CH, pwm_left)

def check_center(change):
    global pwm_center,STEERING_CH
    PCA9685.set_channel_value(STEERING_CH, pwm_center)
    
def check_right(change):
    global pwm_left,STEERING_CH
    PCA9685.set_channel_value(STEERING_CH, pwm_right)
    
load_button.on_click(load_model)
run_button.on_click(run)
stop_button.on_click(stop)
record_box.observe(on_video)
check_left_button.on_click(check_left)
check_center_button.on_click(check_center)
check_right_button.on_click(check_right)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([model_widget,model_time_widget,load_button]),
    ipywidgets.HBox([pwm_left_widget,pwm_center_widget, pwm_right_widget]),
    ipywidgets.HBox([check_left_button,check_center_button,check_right_button]),
    ipywidgets.HBox([pwm_stop_widget, speed_slider]),
    ipywidgets.HBox([name_widget,record_box]),
    ipywidgets.HBox([run_button, stop_button]),
    process_widget
])
display(data_collection_widget)

### カメラの終了処理(必須)

In [ ]:
import time

camera.running = False
camera.cap.release()